In [ ]:
import torch
import random as rdm
import numpy as np
from collections import deque
from Gridworld import Gridworld
from matplotlib import pylab as plt
from IPython.display import clear_output

In [ ]:
# variabeln 
Inputlayer = 64
hiddenlayer1 = 150
hiddenlayer2 = 100
outputlayer = 4
gamma = 0.9
epsilon = 0.3
Lernrate = 1e-3
epochen = 5000 # gibt an wie lange der algorithmus trainieren soll.
testepochen = 3
Gedächnisgrösse = 1000
stapelgrösse = 200 # gibt an wiehoch die menge der zum lernen verwendeten Daten ist bevor der Algorithmus aktualisiert wird.
max_züge = 50

In [ ]:
# dictionarys
aktionsraum = {0:'u', 1:'d', 2: 'l', 3: 'r',}

In [ ]:
# richte neuronales Netz ein.
model = torch.nn.Sequential(torch.nn.Linear(Inputlayer, hiddenlayer1), torch.nn.ReLU(),
                            torch.nn.Linear(hiddenlayer1, hiddenlayer2), torch.nn.ReLU(),
                            torch.nn.Linear(hiddenlayer2, outputlayer))
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=Lernrate)


In [ ]:
losses = []
erinnerungen = deque(maxlen=Gedächnisgrösse) # Erzeugt das Gedächniss 
h = 0

for i in range(epochen):
    # zeigt in welcher Episode wir uns befinden.
    print(i+1)
    clear_output(wait=True)
    # initialisiere Spiel
    game = Gridworld(size=4, mode='random')
    # dem Zustand wird ein kleiner Zufallswert hinzugefügt damit kein Wert genau 0 ist.
    # da viele Funktionen für 0 nicht deffiniert sind kann das ansonsten zu Fehlern führen.
    zustand1 = torch.from_numpy(game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0).float() 
    aktiv = True
    zug = 0
    
    # wird ausgeführ solange das Spiel aktiv ist.
    while(aktiv == True):
        zug += 1
        qwert = model(zustand1) 
        q_wert = qwert.data.numpy()
        # mit der Wahrscheinlichkeit Epsilon wird eine zufällige Aktion gewählt.
        if (rdm.random() < epsilon): 
            aktion = np.random.randint(0,4)
        else:
            aktion = np.argmax(q_wert)
        
        # aktion wird im spiel angewendet
        aktionstring = aktionsraum[aktion] 
        game.makeMove(aktionstring)
        # generiert neuen zustand
        zustand_2 = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0
        zustand2 = torch.from_numpy(zustand_2).float()
        # speichert erfahrungen
        belohnung = game.reward()
        done = True if belohnung > 0 else False # gibt an ob wir für dieses Feld positiv belohnt wurden.
        erfahrung = (zustand1, aktion, belohnung, zustand2, done)
        erinnerungen.append(erfahrung)
        zustand1 = zustand2
        
        # der Q learning Algorithmus wird trainiert wenn genug erinnerungen gesammelt wurden 
        if len(erinnerungen) > stapelgrösse:
            # eine stichprobe wird zum trainieren des Algorithmus bereit gemacht. 
            stichprobe = rdm.sample(erinnerungen, stapelgrösse)
            zustand1_probe = torch.cat([s1 for (s1,a,r,s2,d) in stichprobe])
            zustand2_probe = torch.cat([s2 for (s1,a,r,s2,d) in stichprobe])
            belohnung_probe = torch.Tensor([r for (s1,a,r,s2,d) in stichprobe])
            aktion_probe = torch.Tensor([a for (s1,a,r,s2,d) in stichprobe])
            done_probe = torch.Tensor([d for (s1,a,r,s2,d) in stichprobe])
            # berechnet Q-Wert für die Stichprobe 
            Q_t1 = model(zustand1_probe)
            with torch.no_grad():
                Q_t2 = model(zustand2_probe)
            # Berechnet die Werte die das Netzwerk lernen soll
            Y = belohnung_probe + gamma * ((1 - done_probe) * torch.max(Q_t2,dim=1)[0])
            X = Q_t1.gather(dim=1,index=aktion_probe.long().unsqueeze(dim=1)).squeeze()
            loss = loss_fn(X, Y.detach())
            print(i, loss.item())
            clear_output(wait=True)
            optimizer.zero_grad()
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
                
        if belohnung != -1 or zug >= max_züge: 
            aktiv = False
            zug = 0
losses = np.array(losses)

In [ ]:
# mit dieser Funktion können wir den trainierten Algorithmus testen und sehen wie er Spielt.
for i in range(testepochen):
    test_game = Gridworld(mode="random")
    zustand = torch.from_numpy(test_game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0).float() 
    aktiv = True
    print("neues Spiel")
    zug = 1
    while(aktiv == True):
        qwert = model(zustand) 
        q_wert = qwert.data.numpy()
        aktion = np.argmax(q_wert)
        aktionstring = aktionsraum[aktion] 
        test_game.makeMove(aktionstring)
        print(f" Zug ", zug ,"")
        zug += 1
        print(test_game.display())
        belohnung = test_game.reward()
        zustand = torch.from_numpy(test_game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0).float()
        if belohnung != -1 or zug>20:
            aktiv = False
            if belohnung >= 0:
                print("Gewonnen!")
            else:
                print("verloren :-(")